## Libirary

In [3]:
!pip install langchain_huggingface langchain langchain-community langchain_groq chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 7.9 MB/s eta 0:00:0

In [4]:
!pip install streamlit

In [5]:
import streamlit as st
import requests
from bs4 import BeautifulSoup
import pandas as pd
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
import markdown
import getpass
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

## Web Crawling

In [ ]:
def crawl_contractors(max_pages=10):
    base_url = "https://muqawil.org/en/contractors"
    links = []
    all_contractors = []
    needed_boxes = ["Membership Number" , "Organization Email" , "Organization Mobile Number" , "City"]
    results = {}

    for page in range(1, max_pages + 1):

        url = f"{base_url}?page={page}"

        try:

            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()


            soup = BeautifulSoup(response.content, 'html.parser')


            contractor_cards = soup.find_all('div', class_='has-action')

            print(f"Found {len(contractor_cards)} cards on page {page}")


            for card in contractor_cards:
              # to get the card that have the link to the needed infomation
                botton = card.find('div', class_='col-md-3 col-sm-6')
                link = botton.find('a')['href']
                print(f"Extracted info for {link}")
                links.append(link)
                response = requests.get(link, headers=headers, timeout=10)
                response.raise_for_status()  # Raise exception for bad status codes

                # Parse HTML content
                soup = BeautifulSoup(response.content, 'html.parser')

                # Find contractor cards
                contractor_cards = soup.find_all('div', class_='container')

                print(f"Found {len(contractor_cards)} cards on page {page}")
                cont = 1
                for card in contractor_cards:
                  if cont == 1:
                      title = soup.find('h3', class_='card-title')
                      results['contractor_name'] = title.text.strip()
                      vld_info = soup.find('div', class_='info-box-wrapper')
                      infos = soup.find_all('div', class_='col-md-6 col-lg-4')
                      print(len(infos))
                      # Loop through the info boxes
                      for info in infos:
                          # Find the div with class 'info-name'
                          name_div = info.find('div', class_='info-name')
                          if name_div and name_div.text.strip() in needed_boxes:
                              # Get the corresponding 'info-value'
                              value_div = info.find('div', class_='info-value')
                              if value_div:
                                  # Add to results dictionary
                                  results[name_div.text.strip()] = value_div.text.strip()


                      for key, value in results.items():
                          print(f"{key}: {value}")


                  section_cards = card.find_all('div', class_='section-card')
                  for section in section_cards:
                      title = section.find('h3', class_='card-title')
                      if title and title.text.strip() == 'Interests':

                          interests_text = ' '.join(section.stripped_strings)
                          print(f"Interests Text: {interests_text}")
                          results["Interests"] = interests_text


                  cont = cont + 1
                all_contractors.append(results)





        except requests.RequestException as e:
            print(f"Error crawling page {page}: {e}")
            continue

    # Convert to DataFrame and export to Excel
    df = pd.DataFrame(all_contractors)
    df.to_excel('contractors_data.xlsx', index=False)

    return all_contractors , df

# Run the scraping
contractors , df = crawl_contractors()
print(f"Extracted {len(contractors)} contractor records")


because i want to insert the data into my model i tranfer it into data frame to make it more clear

In [6]:
df

,contractor_name,Membership Number,Organization Mobile Number,Organization Email,City,Interests
0,Alosais Contracting Company,100004776,966138196159,[email protected],DAMMAM,Interests Mining support services Oil and natu...
1,Alosais Contracting Company,100004776,966138196159,[email protected],DAMMAM,Interests Mining support services Oil and natu...
2,Alosais Contracting Company,100004776,966138196159,[email protected],DAMMAM,Interests Mining support services Oil and natu...
3,Alosais Contracting Company,100004776,966138196159,[email protected],DAMMAM,Interests Mining support services Oil and natu...
4,Alosais Contracting Company,100004776,966138196159,[email protected],DAMMAM,Interests Mining support services Oil and natu...
...,...,...,...,...,...,...
195,Alosais Contracting Company,100004776,966138196159,[email protected],DAMMAM,Interests Mining support services Oil and natu...
196,Alosais Contracting Company,100004776,966138196159,[email protected],DAMMAM,Interests Mining support services Oil and natu...
197,Alosais Contracting Company,100004776,966138196159,[email protected],DAMMAM,Interests Mining support services Oil and natu...
198,Alosais Contracting Company,100004776,966138196159,[email protected],DAMMAM,Interests Mining support services Oil and natu...


## XL file

In [7]:
df.to_excel('contractors_data.xlsx', index=False)

## EDA

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Membership Number           20 non-null     object
 1   Organization Mobile Number  20 non-null     object
 2   Organization Email          20 non-null     object
 3   City                        20 non-null     object
 4   contractor_name             20 non-null     object
 5   Interests                   20 non-null     object
dtypes: object(6)
memory usage: 1.1+ KB


In [9]:
df.isna()

,contractor_name,Membership Number,Organization Mobile Number,Organization Email,City,Interests
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False
3,False,False,False,False,False,False
4,False,False,False,False,False,False
...,...,...,...,...,...,...
195,False,False,False,False,False,False
196,False,False,False,False,False,False
197,False,False,False,False,False,False
198,False,False,False,False,False,False


## Embeeding(RAG)

Data processing before embedding

In [12]:
def create_documents(df):
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000000)
    marked_text = []
    for index, row in df.iterrows():
        Membership_Number = row['Membership Number']
        Organization_Mobile_Number = row['Organization Mobile Number']
        Organization_Email = row['Organization Email']
        City = row['City']
        Interests = row['Interests']
        contractor_name = row['contractor_name']

        marked_text.append((f'the information is contractor_name: {contractor_name} ,  Membership_Number: {Membership_Number} , Organization_Mobile_Number : {Organization_Mobile_Number}  , Organization_Email : {Organization_Email} , City : {City}  now the Interests : {Interests}'))
    print(marked_text)
    return splitter.create_documents(marked_text)

Embedding and creating  VDB

In [13]:
def create_embedding(documents):
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return Chroma.from_documents(documents, embeddings)

## LLMs

In [14]:
llm = ChatGroq(temperature=0.8, groq_api_key="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", model_name="llama3-70b-8192")

In [15]:
messages = []

## Prompt

In [16]:
prompt_template = """
<|SYSTEM|>
You are an AI model have information about constaction workers you should give the user what they need form the information you have.
<|END_SYSTEM|>

<|CONTEXT|>
RAG context
{context}

HISTORY context
{history}
<|END_CONTEXT|>

<|USER|>
{query}
<|END_USER|>

<|ASSISTANT|>
"""
prompt_template = PromptTemplate(
    template=prompt_template,
    input_variables=['query' , "context"  , "history" ])

## Model response and RAG context

In [20]:
def generate_answer_response(query, threshold, model):
    results = arabic_VDB.similarity_search_with_score(query, k=1) # i put the k=1 because the interst of each company its too long(because the token numbers is limited on groq)
    context_text = "\n\n".join([doc.page_content for doc, score in results if score > threshold])
    res = MODEL.invoke({"query" : query, "context" : context_text, "history" : messages})
    messages.append({"user" :query, "assistant" : res })
    return  res, context_text

In [21]:
documents = create_documents(df)

['the information is contractor_name: Alosais Contracting Company ,  Membership_Number: 100004776 , Organization_Mobile_Number : 966138196159  , Organization_Email : [email\xa0protected] , City : DAMMAM  now the Interests : Interests Mining support services Oil and natural gas extraction support activities Drilling of oil fields Drilling of natural gas fields Services related to oil extraction except surveying services Services related to natural gas extraction except surveying services Other activities related to oil and natural gas extraction support activities Construction of buildings Construction of buildings Construction of all types of residential buildings Construction of all types of non-residential buildings including schools, hospitals, hotels … etc. Construction of all types of non-residential buildings such as airport buildings Construction of all types of non-residential steel buildings Construction of all types of other not previously mentioned non-residential buildings 

In [22]:
arabic_VDB = create_embedding(documents)
MODEL = LLMChain(llm=llm,
                 prompt=prompt_template,
                 verbose=False)
response , rag = generate_answer_response("give me number for a worker ", 0.8, MODEL)